In [1]:
import os
import joblib
from pathlib import Path
from time import time
import pandas as pd
import numpy as np
from tqdm import tqdm
from model_settings import ms
from quantlib_pricers import barriers

ms.find_root(Path())
ms.__dict__

modelsdir = os.path.join(ms.root,ms.trained_models)
folders = [f for f in os.listdir(modelsdir) if f.find('Legacy')==-1]
folders

['2024-11-15 164145753233 cboe spx relative asian',
 '2024-11-15 164315824655 cboe spx relative barrier']

In [2]:
model_folder = folders[1]
pricename = str(model_folder[model_folder.rfind(' ')+1:])+'_price'
model_dir = os.path.join(modelsdir,model_folder)
pickle = [f for f in os.listdir(model_dir) if f.endswith('.pkl') and f.find(model_folder)!=-1][0]
model = joblib.load(os.path.join(model_dir,pickle))
est = model['model']
df = model['train_data'].iloc[:,:-4].copy().reset_index(drop=True)
df = df.drop(columns=[c for c in df.columns if c.find(pricename)!=-1])
pricename

'barrier_price'

In [6]:
file = [f for f  in os.listdir(Path()) if f.endswith('.csv')][0]

In [15]:
df = pd.read_csv(os.path.join(Path(),file)).iloc[:,1:].copy()
df['MLP'] = df['MLP']*df['strike_price']
df['error'] = df['MLP']-df[pricename]
results = df.iloc[:,-4:].copy()
print(results.describe())

             MLP_cpu  barrier_price    barrier_cpu          error
count  266781.000000  266781.000000  266781.000000  266781.000000
mean        0.002466      75.071199       0.086366       0.009121
std         0.005487     139.369805       0.040133      20.478774
min         0.000000       0.000000       0.029513   -1243.591005
25%         0.000000       0.872015       0.051859      -7.773526
50%         0.000000      22.787113       0.098603       1.654734
75%         0.000000      93.719801       0.110251       7.462184
max         0.091941    1323.022061       0.331892     231.333478


In [18]:
MAE = np.mean(np.abs(df['error']))
RMSE = np.sqrt(np.mean(df['error']**2))

In [19]:
MAE

np.float64(12.914962435003947)

In [20]:
RMSE

np.float64(20.47873765947515)